# Subgroup B: Measuring Campaign ROI

## Exploratory Data Analysis

In this segment, data relevant to addressing the business question is loaded in and prepared for exploratory data analysis.

In [1]:
import  pandas as pd
import numpy as np
import os

# loading in relevant datasets
os.chdir("../updated_data/")
print(os.getcwd())
campaigns = pd.read_csv("campaigns.csv")
customer_engagement = pd.read_csv("customer_engagement.csv")

print(campaigns.head())
print(customer_engagement.head())
print("\n")
print("===========================================================")
print("Information on campaigns dataset")
campaigns.info()
print("\n")
print("===========================================================")
print("Information on customer_engagement dataset")
customer_engagement.info()

C:\Users\rusya\Desktop\NUS\Y3S2\DSA3101\updated_data
   campaign_id     customer_segment  \
0            1              Retired   
1            2        Middle-Market   
2            3              Retired   
3            4     Budget-Conscious   
4            5  Young Professionals   

                                    campaign_name             campaign_type  \
0          Wealth Management Campaign for Retired                     Email   
1            Auto Loan Campaign for Middle-Market  Mobile App Notifications   
2                   Mortgage Campaign for Retired                     Email   
3     Personal Loan Campaign for Budget-Conscious                     Email   
4  Personal Loan Campaign for Young Professionals  Mobile App Notifications   

   start_date    end_date  recommended_product_id recommended_product_name  \
0  2022-03-17  2022-05-04                     107        Wealth Management   
1  2022-03-23  2022-05-02                     106                Auto Loan   
2  

## Feature Engineering
We ensure that columns relevant to answering our business question are created before answering the subquestion in Section B: Measuring Campaign ROI.
In the following code block, the following features are generated: `conversion_rate`, `customer_lifetime_value` and `acquisition_cost`.
`ROI` of past marketing campaigns are also computed, serving as the response variable on which our model will be based on.

### Generating `conversion_rate`

`conversion_rate` is obtained by first determining the number of customers that a campaign has engaged with a customer (i.e number of rows for where `sent` == "Yes", for each campaign), to give us `total_engagements`. This is derived from the `customer_engagement` dataset.

The number of successful conversions is also derived from the `customer_engagement` dataset. `conversion_rate` for each campaign is then computed as follows:

$$
\text{conversion rate} = \frac{\text{successful conversions}}{\text{total engagements}}
$$

In [2]:
# calculating total engagements
total_engagements = customer_engagement[customer_engagement["sent"] == "Yes"].groupby("campaign_id").size().reset_index(name = "total_engagements")
print(total_engagements.head())

# calculating number of successful conversions
successful_conversions = customer_engagement[customer_engagement["conversion_status"] == "Yes"] \
    .groupby("campaign_id").size().reset_index(name="successful_conversions")
print(successful_conversions.head())

# merging engagement data
conversion_data = total_engagements.merge(successful_conversions, on="campaign_id", how="left")
print(conversion_data.head())

# in case no conversions happened for a campaign, fill NA as 0
conversion_data["successful_conversions"] = conversion_data["successful_conversions"].fillna(0)

# computing conversion rate
conversion_data["conversion_rate"] = conversion_data["successful_conversions"] / conversion_data["total_engagements"]
conversion_data.head()

   campaign_id  total_engagements
0            1                473
1            2                740
2            3                946
3            4                570
4            5                829
   campaign_id  successful_conversions
0            1                      52
1            2                      59
2            3                     113
3            4                      48
4            5                      66
   campaign_id  total_engagements  successful_conversions
0            1                473                      52
1            2                740                      59
2            3                946                     113
3            4                570                      48
4            5                829                      66


,campaign_id,total_engagements,successful_conversions,conversion_rate
0,1,473,52,0.109937
1,2,740,59,0.079730
2,3,946,113,0.119450
3,4,570,48,0.084211
4,5,829,66,0.079614


### Generating `acquisition_cost`

`acquisition_cost` is generated with the following formula:

$$
\text{acquisition cost} = \frac{\text{total\_campaign\_cost}}{\text{successful\_conversions}}
$$


In [3]:
acquisition_data = conversion_data.merge(campaigns[["campaign_id","total_campaign_cost"]], on = "campaign_id")
print(acquisition_data.head())

# computing acquisition_cost
acquisition_data["acquisition_cost"] = acquisition_data["total_campaign_cost"] / acquisition_data["successful_conversions"]
acquisition_data.head()

   campaign_id  total_engagements  successful_conversions  conversion_rate  \
0            1                473                      52         0.109937   
1            2                740                      59         0.079730   
2            3                946                     113         0.119450   
3            4                570                      48         0.084211   
4            5                829                      66         0.079614   

   total_campaign_cost  
0             70597.33  
1             87461.14  
2             56053.99  
3             22910.89  
4             24374.42  


,campaign_id,total_engagements,successful_conversions,conversion_rate,total_campaign_cost,acquisition_cost
0,1,473,52,0.109937,70597.33,1357.640962
1,2,740,59,0.079730,87461.14,1482.392203
2,3,946,113,0.119450,56053.99,496.053009
3,4,570,48,0.084211,22910.89,477.310208
4,5,829,66,0.079614,24374.42,369.309394


### Generating `avg_targeted_clv`

`customer_lifetime_value` will first be attached to each individual customer in the `customers` dataset.

In the following code block, a customer's average transaction value is calculated based on the `total_transaction_amount_12m`.

In [4]:
# loading in customer and transaction data
customers = pd.read_csv("customers.csv")
transactions = pd.read_csv("transactions_summary.csv")

# merging customer and transaction data
customers = customers.merge(transactions, on = "customer_id", how = "left")

To estimate customer_lifetime, I will be using a simplified model. 

`churn_prob` is estimated using the `churn threshold` of 90 days without any transaction, from the `days_since_last_transaction` column. A value of 0.8 will be assigned to customers who have not interacted with the bank for a long period of time and 0.2 will be assigned to customers who are still interacting with the bank relatively frequently.

`estimated_lifetime` for each customer will be calculated using the following formula:

$$
\text{Estimated Customer Lifetime} = \frac{\text{1}}{\text{Churn Probability}}
$$

Finally, `CLV` is computed by multiplying `expected_lifetime` with `average_transaction_amount_12m`

In [5]:
# assigning churn probability to each customer
churn_threshold = 90
customers["churn_prob"] = np.where(customers["days_since_last_transaction"] > churn_threshold, 0.8, 0.2)

# computing estimated lifetime
customers["expected_lifetime"] = 1/customers["churn_prob"]

# computing CLV for each customer
customers["CLV"] = customers["expected_lifetime"] * customers ["average_transaction_amount_12m"]

customers["CLV"].head()

0     702.173803
1     133.391687
2     572.853830
3     458.526423
4    1577.008438
Name: CLV, dtype: float64

I then generate `avg_targeted_clv` for each campaign.

In [6]:
# merging CLV data with customer engagement to get CLV of targeted customers
engagement_clv = customer_engagement.merge(customers[["customer_id","CLV"]], on = "customer_id", how = "left")

# grouping by campaign ID
campaign_clv = engagement_clv.groupby("campaign_id")["CLV"].mean().reset_index()
campaign_clv.rename(columns = {"CLV": "avg_targeted_clv"}, inplace = True)
campaign_clv.head()

,campaign_id,avg_targeted_clv
0,1,927.588803
1,2,889.239655
2,3,903.921540
3,4,905.067445
4,5,945.753058


### Generating `ROI` for past marketing campaigns

ROI is calculated using data from the `campaigns` dataset. The formula used is as follows:

$$
\text{ROI} = \frac{\text{revenue} - \text{cost}}{\text{cost}}
$$

In [14]:
roi_data = campaigns[["campaign_id","total_campaign_cost","total_revenue_generated"]]
roi_data["roi"] = (roi_data["total_revenue_generated"] - roi_data["total_campaign_cost"])/roi_data["total_campaign_cost"]
roi_data.head()

C:\Users\rusya\AppData\Local\Temp\ipykernel_23548\2772495362.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  roi_data["roi"] = (roi_data["total_revenue_generated"] - roi_data["total_campaign_cost"])/roi_data["total_campaign_cost"]


,campaign_id,total_campaign_cost,total_revenue_generated,roi
0,1,70597.33,94151.78,0.333645
1,2,87461.14,184133.52,1.105318
2,3,56053.99,160828.61,1.869173
3,4,22910.89,58342.54,1.546498
4,5,24374.42,63738.91,1.614992


## Data Preparation for Machine Learning

Here, we further manipulate and select features from the `campaigns` dataset with all the feature engineered features above. This dataset will be used for model training.

In [26]:
## manipulation of dataframes to create X
from sklearn.preprocessing import OneHotEncoder

# One-hot encode campaign_type
campaigns_encoded = pd.get_dummies(campaigns, columns = ["campaign_type"], drop_first = True)

# One-hot encode product
campaigns_encoded.rename(columns = {"recommended_product_name": "product"}, inplace = True)
campaigns_encoded = pd.get_dummies(campaigns_encoded, columns=['product'], drop_first=True)

# One-hot encode customer_segment
campaigns_encoded = pd.get_dummies(campaigns_encoded, columns = ['customer_segment'], drop_first = True)
print(campaigns_encoded.head())

# compute campaign_duration
campaigns_encoded["start_date"] = pd.to_datetime(campaigns_encoded["start_date"])
campaigns_encoded["end_date"] = pd.to_datetime(campaigns_encoded["end_date"])

campaigns_encoded["campaign_duration"] = (campaigns_encoded["end_date"] - campaigns_encoded["start_date"]).dt.days

# merging with feature engineered columns
campaigns_encoded = campaigns_encoded.merge(conversion_data[["campaign_id","conversion_rate"]], on = "campaign_id", how = "left")
campaigns_encoded = campaigns_encoded.merge(acquisition_data[["campaign_id","acquisition_cost"]], on = "campaign_id", how = "left")
campaigns_encoded = campaigns_encoded.merge(campaign_clv[["campaign_id","avg_targeted_clv"]], on = "campaign_id", how = "left")

campaigns_encoded.columns


   campaign_id                                   campaign_name  start_date  \
0            1          Wealth Management Campaign for Retired  2022-03-17   
1            2            Auto Loan Campaign for Middle-Market  2022-03-23   
2            3                   Mortgage Campaign for Retired  2022-05-22   
3            4     Personal Loan Campaign for Budget-Conscious  2023-01-13   
4            5  Personal Loan Campaign for Young Professionals  2023-12-06   

     end_date  recommended_product_id  campaign_budget  total_campaign_cost  \
0  2022-05-04                     107         58055.93             70597.33   
1  2022-05-02                     106         35527.89             87461.14   
2  2022-06-26                     103         52445.28             56053.99   
3  2023-02-21                     102         57816.78             22910.89   
4  2024-01-13                     102         36979.25             24374.42   

   total_revenue_generated  campaign_type_Email  \
0    

Index(['campaign_id', 'campaign_name', 'start_date', 'end_date',
       'recommended_product_id', 'campaign_budget', 'total_campaign_cost',
       'total_revenue_generated', 'campaign_type_Email',
       'campaign_type_Mobile App Notifications', 'campaign_type_SMS',
       'product_Credit Card', 'product_Investment Product', 'product_Mortgage',
       'product_Personal Loan', 'product_Savings Account',
       'product_Wealth Management', 'customer_segment_High-Value',
       'customer_segment_Middle-Market', 'customer_segment_Retired',
       'customer_segment_Young Professionals', 'campaign_duration',
       'conversion_rate', 'acquisition_cost', 'avg_targeted_clv'],
      dtype='object')

In [102]:
features = ["total_campaign_cost", "campaign_duration",
       'conversion_rate', 'acquisition_cost', 'avg_targeted_clv']
customer_segments = ['customer_segment_High-Value',
       'customer_segment_Middle-Market', 'customer_segment_Retired',
       'customer_segment_Young Professionals']
products = ['product_Credit Card', 'product_Investment Product', 'product_Mortgage',
       'product_Personal Loan', 'product_Savings Account',
       'product_Wealth Management']
campaign_types = ['campaign_type_Email',
       'campaign_type_Mobile App Notifications', 'campaign_type_SMS']

selected_features = features

## Model Implementation (Random Forests)


In [104]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# selecting features, creating target variable
X = campaigns_encoded[selected_features]
y = roi_data["roi"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize model
model = RandomForestRegressor(n_estimators=50, random_state=42)

# Train model
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print(mae)
r2 = r2_score(y_test, y_pred)
print(r2)

0.47073164287851704
-0.33115492761937215


The model is performing very poorly. Some troubleshooting:

In [97]:
feature_importances = pd.Series(model.feature_importances_, index=selected_features)
print(feature_importances.sort_values(ascending=False))

total_campaign_cost                       0.213285
conversion_rate                           0.165413
avg_targeted_clv                          0.162451
campaign_duration                         0.133169
acquisition_cost                          0.114688
campaign_type_Mobile App Notifications    0.054821
customer_segment_Retired                  0.048003
campaign_type_SMS                         0.018856
product_Mortgage                          0.015572
customer_segment_Young Professionals      0.012021
customer_segment_Middle-Market            0.010495
product_Personal Loan                     0.009862
product_Wealth Management                 0.009841
product_Credit Card                       0.007866
customer_segment_High-Value               0.007281
product_Investment Product                0.005565
product_Savings Account                   0.005438
campaign_type_Email                       0.005373
dtype: float64


Removing features with importance less than 0.01:

In [98]:
selected_features = feature_importances[feature_importances > 0.01].index.tolist()
selected_features

['total_campaign_cost',
 'campaign_duration',
 'conversion_rate',
 'acquisition_cost',
 'avg_targeted_clv',
 'customer_segment_Middle-Market',
 'customer_segment_Retired',
 'customer_segment_Young Professionals',
 'product_Mortgage',
 'campaign_type_Mobile App Notifications',
 'campaign_type_SMS']

In [101]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Define hyperparameter grid (smaller range than RandomizedSearchCV for efficiency)
param_grid = {
    "n_estimators": [100, 200, 300],  # Number of trees
    "max_depth": [10, 20, None],  # Tree depth
    "min_samples_split": [2, 5],  # Min samples to split a node
    "min_samples_leaf": [1, 2],  # Min samples at a leaf node
    "max_features": ["sqrt", "log2"],  # Number of features per split
    "bootstrap": [True, False]  # Whether to use bootstrapping
}

# Initialize RandomForestRegressor
rf = RandomForestRegressor(random_state=1)

# Perform Grid Search with 5-fold Cross-Validation
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available CPU cores
)

# Fit to training data
grid_search.fit(X_train, y_train)

# Best hyperparameters
print("Best Parameters:", grid_search.best_params_)

# Use best model
best_rf = grid_search.best_estimator_

# Predict on test set
y_pred = best_rf.predict(X_test)

# Evaluate performance
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Tuned Model MAE:", mae)
print("Tuned Model R²:", r2)


Fitting 5 folds for each of 144 candidates, totalling 720 fits
Best Parameters: {'bootstrap': True, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 300}
Tuned Model MAE: 0.5341973191474929
Tuned Model R²: -0.6061821357528767


In [85]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=1)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

print("XGBoost R²:", r2_score(y_test, y_pred_xgb))
print("XGBoost MAE:", mean_absolute_error(y_test, y_pred_xgb))


XGBoost R²: -1.139314371935932
XGBoost MAE: 0.6396177241726749
